In [4]:
%iam_role arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service"

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is None
iam_role has been set to arn:aws:iam::041651352119:role/tgedr_it_role_for_glue_service.


In [1]:
spark

Trying to create a Glue session for the kernel.
Session Type: etl
Worker Type: G.1X
Number of Workers: 5
Session ID: 0cdb4901-3942-46ff-b5a7-3eefc949551b
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
Waiting for session 0cdb4901-3942-46ff-b5a7-3eefc949551b to get into ready status...
Session 0cdb4901-3942-46ff-b5a7-3eefc949551b has been created.
